In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import requests as req
from bs4 import BeautifulSoup as BS


In [3]:
test_dot = pd.read_csv("test_dot.csv")

In [34]:
origin = test_dot[["Origin_Airport_ID", "Origin_State"]]
origin = origin.dropna()
origin_dict = dict(origin.drop_duplicates().values)

In [35]:
origin_2 = test_dot[["Destination_Airport_ID", "Destination_State"]]
origin_2 = origin_2.dropna()
origin_dict_2 = dict(origin_2.drop_duplicates().values)

In [36]:
origin_dict[14534] = 'Wyoming'
origin_dict[14812] = 'Wyoming'
origin_dict[11283] = 'Kansas'
origin_dict[11288] = 'Illinois'

origin_dict_2[14534] = 'Wyoming'
origin_dict_2[14812] = 'Wyoming'
origin_dict_2[11283] = 'Kansas'
origin_dict_2[11288] = 'Illinois'


In [37]:
for index, row in test_dot.iterrows():
    if pd.isna(row["Origin_State"]):
        test_dot.at[index, "Origin_State"] = origin_dict.get(row["Origin_Airport_ID"], None)

In [38]:
for index, row in test_dot.iterrows():
    if pd.isna(row["Destination_State"]):
        test_dot.at[index, "Destination_State"] = origin_dict.get(row["Destination_Airport_ID"], None)

In [39]:
test_dot[test_dot['Origin_State'].isnull()]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,tail_alpha


In [40]:
test_dot[test_dot['Destination_State'].isnull()]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,tail_alpha


In [41]:
states = list(test_dot['Origin_State'].unique())

In [45]:
states = list(test_dot['Origin_State'].unique())
states_dict = {}

for state in states:
    url = f'https://ko.thetimenow.com/united_states/{state}'
    res = req.get(url)
    soup = BS(res.text, "html.parser")
    time_tags = soup.select('body > div.fixed-wraper > section:nth-child(2) > div > article.half-block.clearrm > div > section > div:nth-child(2) > div:nth-child(1)')
    offset = ''
    for tag in time_tags:
        offset += tag.text.strip()
    states_dict[state] = offset

print(states_dict)

{'Texas': 'UTC/GMT -6시간', 'New Jersey': 'UTC/GMT -5시간', 'Illinois': 'UTC/GMT -6시간', 'California': 'UTC/GMT -8시간', 'Florida': 'UTC/GMT -5시간', 'Virginia': 'UTC/GMT -5시간', 'Tennessee': 'UTC/GMT -6시간', 'Missouri': 'UTC/GMT -6시간', 'Arizona': 'UTC/GMT -7시간', 'Colorado': 'UTC/GMT -7시간', 'Nevada': 'UTC/GMT -8시간', 'Ohio': 'UTC/GMT -5시간', 'Kentucky': 'UTC/GMT -5시간', 'Michigan': 'UTC/GMT -5시간', 'Georgia': 'UTC/GMT -5시간', 'Alaska': 'UTC/GMT -9시간', 'Washington': 'UTC/GMT -8시간', 'Hawaii': 'UTC/GMT -10시간', 'Pennsylvania': 'UTC/GMT -5시간', 'North Carolina': 'UTC/GMT -5시간', 'Maryland': 'UTC/GMT -5시간', 'Arkansas': 'UTC/GMT -6시간', 'Nebraska': 'UTC/GMT -6시간', 'New York': 'UTC/GMT -5시간', 'Rhode Island': 'UTC/GMT -5시간', 'Minnesota': 'UTC/GMT -6시간', 'Vermont': 'UTC/GMT -5시간', 'Louisiana': 'UTC/GMT -6시간', 'Massachusetts': 'UTC/GMT -5시간', 'South Carolina': 'UTC/GMT -5시간', 'Montana': 'UTC/GMT -7시간', 'New Hampshire': 'UTC/GMT -5시간', 'Alabama': 'UTC/GMT -6시간', 'Oklahoma': 'UTC/GMT -6시간', 'Puerto Rico': '', 'Idaho'

In [46]:
states_dict

{'Texas': 'UTC/GMT -6시간',
 'New Jersey': 'UTC/GMT -5시간',
 'Illinois': 'UTC/GMT -6시간',
 'California': 'UTC/GMT -8시간',
 'Florida': 'UTC/GMT -5시간',
 'Virginia': 'UTC/GMT -5시간',
 'Tennessee': 'UTC/GMT -6시간',
 'Missouri': 'UTC/GMT -6시간',
 'Arizona': 'UTC/GMT -7시간',
 'Colorado': 'UTC/GMT -7시간',
 'Nevada': 'UTC/GMT -8시간',
 'Ohio': 'UTC/GMT -5시간',
 'Kentucky': 'UTC/GMT -5시간',
 'Michigan': 'UTC/GMT -5시간',
 'Georgia': 'UTC/GMT -5시간',
 'Alaska': 'UTC/GMT -9시간',
 'Washington': 'UTC/GMT -8시간',
 'Hawaii': 'UTC/GMT -10시간',
 'Pennsylvania': 'UTC/GMT -5시간',
 'North Carolina': 'UTC/GMT -5시간',
 'Maryland': 'UTC/GMT -5시간',
 'Arkansas': 'UTC/GMT -6시간',
 'Nebraska': 'UTC/GMT -6시간',
 'New York': 'UTC/GMT -5시간',
 'Rhode Island': 'UTC/GMT -5시간',
 'Minnesota': 'UTC/GMT -6시간',
 'Vermont': 'UTC/GMT -5시간',
 'Louisiana': 'UTC/GMT -6시간',
 'Massachusetts': 'UTC/GMT -5시간',
 'South Carolina': 'UTC/GMT -5시간',
 'Montana': 'UTC/GMT -7시간',
 'New Hampshire': 'UTC/GMT -5시간',
 'Alabama': 'UTC/GMT -6시간',
 'Oklahoma': 'UTC/GMT 

In [47]:
empty_values = [key for key, value in states_dict.items() if not value]
print(empty_values)

['Puerto Rico', 'U.S. Virgin Islands', 'U.S. Pacific Trust Territories and Possessions']


In [48]:
states_dict['Puerto Rico'] = 'UTC/GMT -4시간'
states_dict['U.S. Virgin Islands'] = "UTC/GMT -4시간"

In [49]:
del states_dict['U.S. Pacific Trust Territories and Possessions']

In [50]:
for state, offset_str in states_dict.items():
    offset = int(offset_str.split()[1].replace('시간', ''))
    states_dict[state] = str(offset)

print(states_dict)

{'Texas': '-6', 'New Jersey': '-5', 'Illinois': '-6', 'California': '-8', 'Florida': '-5', 'Virginia': '-5', 'Tennessee': '-6', 'Missouri': '-6', 'Arizona': '-7', 'Colorado': '-7', 'Nevada': '-8', 'Ohio': '-5', 'Kentucky': '-5', 'Michigan': '-5', 'Georgia': '-5', 'Alaska': '-9', 'Washington': '-8', 'Hawaii': '-10', 'Pennsylvania': '-5', 'North Carolina': '-5', 'Maryland': '-5', 'Arkansas': '-6', 'Nebraska': '-6', 'New York': '-5', 'Rhode Island': '-5', 'Minnesota': '-6', 'Vermont': '-5', 'Louisiana': '-6', 'Massachusetts': '-5', 'South Carolina': '-5', 'Montana': '-7', 'New Hampshire': '-5', 'Alabama': '-6', 'Oklahoma': '-6', 'Puerto Rico': '-4', 'Idaho': '-7', 'Indiana': '-5', 'Connecticut': '-5', 'Wisconsin': '-6', 'Utah': '-7', 'Oregon': '-8', 'New Mexico': '-7', 'Maine': '-5', 'Iowa': '-6', 'South Dakota': '-6', 'North Dakota': '-6', 'West Virginia': '-5', 'Wyoming': '-7', 'U.S. Virgin Islands': '-4', 'Kansas': '-6', 'Mississippi': '-6'}


In [51]:
test_dot['Estimated_Departure_Time_Zone'] = None
test_dot["Estimated_Arrival_Time_Zone"] = None

for index, row in test_dot.iterrows():
    if row["Origin_State"] == 'U.S. Pacific Trust Territories and Possessions':
        if row['Origin_Airport'] == 'HNL':
            test_dot.at[index, "Estimated_Departure_Time_Zone"] = '-10'
        else:
            test_dot.at[index, "Estimated_Departure_Time_Zone"] = '10'
    else:
        test_dot.at[index, "Estimated_Departure_Time_Zone"] = states_dict.get(row['Origin_State'])
        
    if row["Destination_State"] == 'U.S. Pacific Trust Territories and Possessions':
        if row['Destination_Airport'] == 'HNL':
            test_dot.at[index, "Estimated_Arrival_Time_Zone"] = '-10'
        else:
            test_dot.at[index, "Estimated_Arrival_Time_Zone"] = '10'
    else:
        test_dot.at[index, "Estimated_Arrival_Time_Zone"] = states_dict.get(row['Destination_State'])


In [54]:
test_dot["Estimated_Departure_Time_Zone"] = test_dot["Estimated_Departure_Time_Zone"].astype(int)
test_dot["Estimated_Arrival_Time_Zone"] = test_dot["Estimated_Arrival_Time_Zone"].astype(int)

In [53]:
test_dot.to_csv("test_dot_timezone.csv", index =False)